## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### Process Data

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7f8f3c73fb50>)

In [4]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

NameError: name 'HYPOTHETICAL_SUBMISSION_DATE' is not defined

In [ ]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [ ]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [ ]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [ ]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [ ]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [ ]:
# Only look at US regions
df = df[(df['CountryName'] == 'United States') & (df['RegionName'].notnull())]

In [ ]:
df

### Model Training

In [ ]:
def process_data(df, nb_lookback_days, nb_lookback_months=20, test_months=2):    
    # Create training data across all regions for predicting one day ahead
    X_cols = cases_col + npi_cols
    y_cols = ['GeoID', 'Date', 'NewCases'] # Include GeoID and date to create graphs
    X_samples = []
    y_samples = []
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    train_date = SUBMISSION_DATE-np.timedelta64((nb_lookback_months+test_months)*30, 'D')
    geo_ids = df.GeoID.unique()
    for g in geo_ids:
        gdf = df[(df.GeoID == g) & (df.Date >= train_date)]
        all_case_data = np.array(gdf[cases_col])
        all_npi_data = np.array(gdf[npi_cols])
        all_bookkeeping_data = np.array(gdf[y_cols])

        # Create one sample for each day where we have enough data
        # Each sample consists of cases and npis for previous nb_lookback_days
        nb_total_days = len(gdf)
        
        for d in range(nb_lookback_days, nb_total_days - 1):
            X_cases = all_case_data[d-nb_lookback_days:d]
            X_npis = all_npi_data[d - nb_lookback_days:d]

            # Flatten all input data
            X_sample = np.concatenate([X_cases.flatten(),
                                       X_npis.flatten()])
            y_sample = all_bookkeeping_data[d]
            
            if d < nb_total_days-test_months*30:
                X_train.append(X_sample)
                y_train.append(y_sample)
            else:
                X_test.append(X_sample)
                y_test.append(y_sample)
    
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)[:,-1]
    y_test = np.array(y_test)
    return X_train, X_test, y_train, y_test

In [ ]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [ ]:
def run(df, nb_lookback_days, nb_lookback_months, test_months, model, inspect=False, graph=False):
    df_copy = df.copy()
    X_train, X_test, y_train, y_test = process_data(df_copy,
                                                    nb_lookback_days, 
                                                    nb_lookback_months,
                                                    test_months
                                                   )
    model.fit(X_train, y_train)
    # Evaluate model
    train_preds = model.predict(X_train)
    train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
    print('Train MAE:', mae(train_preds, y_train))
    test_preds = model.predict(X_test)
    test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
    print('Test MAE:', mae(test_preds, y_test[:,-1]))
    
    # Inspect the learned feature coefficients for the model
    # to see what features it's paying attention to.
    if inspect:
        # Give names to the features
        x_col_names = []
        for d in range(-nb_lookback_days, 0):
            x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
        for d in range(-nb_lookback_days, 1):
            for col_name in npi_cols:
                x_col_names.append('Day ' + str(d) + ' ' + col_name)

        # View non-zero coefficients
        for (col, coeff) in zip(x_col_names, list(model.coef_)):
            if coeff != 0.:
                print(col, coeff)
        print('Intercept', model.intercept_)
        
    if graph:
        graph_np = np.hstack((y_test, test_preds.reshape(-1,1)))
        geo_ids = np.unique(graph_np[:,0])
        for geo_id in geo_ids:
            gnp = graph_np[graph_np[:,0]==geo_id, :]
            plt.figure()
            plt.plot(gnp[:,1], gnp[:,3], label='Prediction')
            plt.plot(gnp[:,1], gnp[:,2], label='Actual')
            plt.title(geo_id)
            plt.legend()
            plt.show()
            
    return model


In [ ]:
model = run(df, 30, 20, 2, LinearRegression(), False, False)

In [ ]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [10]:
MODEL_FILE = "./models/model.pkl"
DATA_FILE = "./data/OxCGRT_latest.csv"
ID_COLS = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
CASES_COL = ['NewCases']
NPI_COLS = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
NB_LOOKBACK_DAYS = 30

In [11]:
def predict(start_date: str,
            end_date: str,
            path_to_ips_file: str,
            output_file_path) -> None:
    """
    Generates and saves a file with daily new cases predictions for the given countries, regions and intervention
    plans, between start_date and end_date, included.
    :param start_date: day from which to start making predictions, as a string, format YYYY-MM-DDD
    :param end_date: day on which to stop making predictions, as a string, format YYYY-MM-DDD
    :param path_to_ips_file: path to a csv file containing the intervention plans between inception date (Jan 1 2020)
     and end_date, for the countries and regions for which a prediction is needed
    :param output_file_path: path to file to save the predictions to
    :return: Nothing. Saves the generated predictions to an output_file_path CSV file
    with columns "CountryName,RegionName,Date,PredictedDailyNewCases"
    """
    preds_df = predict_df(start_date, end_date, path_to_ips_file, verbose=False)
    # Create the output path
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    # Save to a csv file
    preds_df.to_csv(output_file_path, index=False)
    print(f"Saved predictions to {output_file_path}")

In [14]:
def predict_df(start_date_str: str, end_date_str: str, path_to_ips_file: str, verbose=False):
    """
    Generates a file with daily new cases predictions for the given countries, regions and npis, between
    start_date and end_date, included.
    :param start_date_str: day from which to start making predictions, as a string, format YYYY-MM-DDD
    :param end_date_str: day on which to stop making predictions, as a string, format YYYY-MM-DDD
    :param path_to_ips_file: path to a csv file containing the intervention plans between inception_date and end_date
    :param verbose: True to print debug logs
    :return: a Pandas DataFrame containing the predictions
    """
    start_date = pd.to_datetime(start_date_str, format='%Y-%m-%d')
    end_date = pd.to_datetime(end_date_str, format='%Y-%m-%d')

    # Load historical intervention plans, since inception
    hist_ips_df = pd.read_csv(path_to_ips_file,
                              parse_dates=['Date'],
                              encoding="ISO-8859-1",
                              dtype={"RegionName": str},
                              error_bad_lines=True)

    # Add GeoID column that combines CountryName and RegionName for easier manipulation of data",
    hist_ips_df['GeoID'] = hist_ips_df['CountryName'] + '__' + hist_ips_df['RegionName'].astype(str)
    # Fill any missing NPIs by assuming they are the same as previous day
    for npi_col in NPI_COLS:
        hist_ips_df.update(hist_ips_df.groupby(['CountryName', 'RegionName'])[npi_col].ffill().fillna(0))

    # Intervention plans to forecast for: those between start_date and end_date
    ips_df = hist_ips_df[(hist_ips_df.Date >= start_date) & (hist_ips_df.Date <= end_date)]

    # Load historical data to use in making predictions in the same way
    # This is the data we trained on
    # We stored it locally as for predictions there will be no access to the internet
    hist_cases_df = pd.read_csv(DATA_FILE,
                                parse_dates=['Date'],
                                encoding="ISO-8859-1",
                                dtype={"RegionName": str,
                                       "RegionCode": str},
                                error_bad_lines=False)
    # Add RegionID column that combines CountryName and RegionName for easier manipulation of data
    hist_cases_df['GeoID'] = hist_cases_df['CountryName'] + '__' + hist_cases_df['RegionName'].astype(str)
    # Add new cases column
    hist_cases_df['NewCases'] = hist_cases_df.groupby('GeoID').ConfirmedCases.diff().fillna(0)
    # Fill any missing case values by interpolation and setting NaNs to 0
    hist_cases_df.update(hist_cases_df.groupby('GeoID').NewCases.apply(
        lambda group: group.interpolate()).fillna(0))
    # Keep only the id and cases columns
    hist_cases_df = hist_cases_df[ID_COLS + CASES_COL]

    # Load model
    with open(MODEL_FILE, 'rb') as model_file:
        model = pickle.load(model_file)

    # Make predictions for each country,region pair
    geo_pred_dfs = []
    for g in ips_df.GeoID.unique():
        if verbose:
            print('\nPredicting for', g)

        # Pull out all relevant data for country c
        # Start predicting from start_date, unless there's a gap since last known date
        hist_cases_gdf = hist_cases_df.loc[hist_cases_df.GeoID == g]
        last_known_date = hist_cases_gdf.Date.max()        
        current_date = min(last_known_date + np.timedelta64(1, 'D'), start_date)
        hist_cases_gdf = hist_cases_gdf.loc[hist_cases_df.Date < current_date]
        hist_ips_gdf = hist_ips_df[(hist_ips_df.GeoID == g) & (hist_ips_df.Date < current_date)]
        ips_gdf = ips_df[ips_df.GeoID == g]
        
        past_cases = np.array(hist_cases_gdf[CASES_COL])
        past_npis = np.array(hist_ips_gdf[NPI_COLS])
        future_npis = np.array(ips_gdf[NPI_COLS])
        
        # Make prediction for each day
        geo_preds = []
        days_ahead = 0
        while current_date <= end_date:
            # Prepare data
            X_cases = past_cases[-NB_LOOKBACK_DAYS:]
            X_npis = past_npis[-NB_LOOKBACK_DAYS:]
            X = np.concatenate([X_cases.flatten(),
                                X_npis.flatten()])

            # Make the prediction (reshape so that sklearn is happy)
            pred = model.predict(X.reshape(1, -1))[0]
            pred = max(0, pred)  # Do not allow predicting negative cases
            # Add if it's a requested date
            if current_date >= start_date:
                geo_preds.append(pred)
                if verbose:
                    print(f"{current_date.strftime('%Y-%m-%d')}: {pred}")
            else:
                if verbose:
                    print(f"{current_date.strftime('%Y-%m-%d')}: {pred} - Skipped (intermediate missing daily cases)")

            # Append the prediction and npi's for next day
            # in order to rollout predictions for further days.
            past_cases = np.append(past_cases, pred)
            past_npis = np.append(past_npis, future_npis[days_ahead:days_ahead + 1], axis=0)

            # Move to next day
            current_date = current_date + np.timedelta64(1, 'D')
            days_ahead += 1

        # Create geo_pred_df with pred column
        geo_pred_df = ips_gdf[ID_COLS].copy()
        geo_pred_df['PredictedDailyNewCases'] = geo_preds
        geo_pred_dfs.append(geo_pred_df)
        
        if 'United States' in g:
            print(geo_pred_df)

    # Combine all predictions into a single dataframe
    pred_df = pd.concat(geo_pred_dfs)

    # Drop GeoID column to match expected output format
    pred_df = pred_df.drop(columns=['GeoID'])
    return pred_df

In [15]:
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=False)

         CountryName RegionName               GeoID       Date  \
55835  United States        NaN  United States__nan 2020-08-01   
55836  United States        NaN  United States__nan 2020-08-02   
55837  United States        NaN  United States__nan 2020-08-03   
55838  United States        NaN  United States__nan 2020-08-04   
55839  United States        NaN  United States__nan 2020-08-05   
55840  United States        NaN  United States__nan 2020-08-06   
55841  United States        NaN  United States__nan 2020-08-07   
55842  United States        NaN  United States__nan 2020-08-08   
55843  United States        NaN  United States__nan 2020-08-09   
55844  United States        NaN  United States__nan 2020-08-10   
55845  United States        NaN  United States__nan 2020-08-11   
55846  United States        NaN  United States__nan 2020-08-12   
55847  United States        NaN  United States__nan 2020-08-13   
55848  United States        NaN  United States__nan 2020-08-14   
55849  Uni

         CountryName RegionName                  GeoID       Date  \
59123  United States     Hawaii  United States__Hawaii 2020-08-01   
59124  United States     Hawaii  United States__Hawaii 2020-08-02   
59125  United States     Hawaii  United States__Hawaii 2020-08-03   
59126  United States     Hawaii  United States__Hawaii 2020-08-04   
59127  United States     Hawaii  United States__Hawaii 2020-08-05   
59128  United States     Hawaii  United States__Hawaii 2020-08-06   
59129  United States     Hawaii  United States__Hawaii 2020-08-07   
59130  United States     Hawaii  United States__Hawaii 2020-08-08   
59131  United States     Hawaii  United States__Hawaii 2020-08-09   
59132  United States     Hawaii  United States__Hawaii 2020-08-10   
59133  United States     Hawaii  United States__Hawaii 2020-08-11   
59134  United States     Hawaii  United States__Hawaii 2020-08-12   
59135  United States     Hawaii  United States__Hawaii 2020-08-13   
59136  United States     Hawaii  U

         CountryName RegionName                    GeoID       Date  \
62137  United States   Michigan  United States__Michigan 2020-08-01   
62138  United States   Michigan  United States__Michigan 2020-08-02   
62139  United States   Michigan  United States__Michigan 2020-08-03   
62140  United States   Michigan  United States__Michigan 2020-08-04   
62141  United States   Michigan  United States__Michigan 2020-08-05   
62142  United States   Michigan  United States__Michigan 2020-08-06   
62143  United States   Michigan  United States__Michigan 2020-08-07   
62144  United States   Michigan  United States__Michigan 2020-08-08   
62145  United States   Michigan  United States__Michigan 2020-08-09   
62146  United States   Michigan  United States__Michigan 2020-08-10   
62147  United States   Michigan  United States__Michigan 2020-08-11   
62148  United States   Michigan  United States__Michigan 2020-08-12   
62149  United States   Michigan  United States__Michigan 2020-08-13   
62150 

         CountryName RegionName                GeoID       Date  \
65699  United States       Ohio  United States__Ohio 2020-08-01   
65700  United States       Ohio  United States__Ohio 2020-08-02   
65701  United States       Ohio  United States__Ohio 2020-08-03   
65702  United States       Ohio  United States__Ohio 2020-08-04   
65703  United States       Ohio  United States__Ohio 2020-08-05   
65704  United States       Ohio  United States__Ohio 2020-08-06   
65705  United States       Ohio  United States__Ohio 2020-08-07   
65706  United States       Ohio  United States__Ohio 2020-08-08   
65707  United States       Ohio  United States__Ohio 2020-08-09   
65708  United States       Ohio  United States__Ohio 2020-08-10   
65709  United States       Ohio  United States__Ohio 2020-08-11   
65710  United States       Ohio  United States__Ohio 2020-08-12   
65711  United States       Ohio  United States__Ohio 2020-08-13   
65712  United States       Ohio  United States__Ohio 2020-08-1

         CountryName RegionName                   GeoID       Date  \
68987  United States    Vermont  United States__Vermont 2020-08-01   
68988  United States    Vermont  United States__Vermont 2020-08-02   
68989  United States    Vermont  United States__Vermont 2020-08-03   
68990  United States    Vermont  United States__Vermont 2020-08-04   
68991  United States    Vermont  United States__Vermont 2020-08-05   
68992  United States    Vermont  United States__Vermont 2020-08-06   
68993  United States    Vermont  United States__Vermont 2020-08-07   
68994  United States    Vermont  United States__Vermont 2020-08-08   
68995  United States    Vermont  United States__Vermont 2020-08-09   
68996  United States    Vermont  United States__Vermont 2020-08-10   
68997  United States    Vermont  United States__Vermont 2020-08-11   
68998  United States    Vermont  United States__Vermont 2020-08-12   
68999  United States    Vermont  United States__Vermont 2020-08-13   
69000  United States

In [16]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [200]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 0
2020-08-02: 39.63190557658831
2020-08-03: 35.08006232688008
2020-08-04: 0
2020-08-05: 0
2020-08-06: 140.1772472822075
2020-08-07: 66.1162409344055
2020-08-08: 3.512045879819212
2020-08-09: 0
2020-08-10: 79.36926246222464
2020-08-11: 97.54936675661153
2020-08-12: 317.2590807925048
2020-08-13: 160.25142584745691
2020-08-14: 138.7234234587866
2020-08-15: 146.98884297377924
2020-08-16: 76.49620246217773
2020-08-17: 248.22511521603275
2020-08-18: 224.83929201750988
2020-08-19: 172.17630388934217
2020-08-20: 215.67901627834047
2020-08-21: 205.69538174941965
2020-08-22: 208.272787830014
2020-08-23: 99.66771402048514
2020-08-24: 144.67288120916353
2020-08-25: 88.62475929963765
2020-08-26: 258.37679420619287
2020-08-27: 149.35365000453015
2020-08-28: 213.26765910966319
2020-08-29: 155.64534611886097
2020-08-30: 191.83852627846306
2020-08-31: 176.98821081012886

Predicting for Afghanistan__nan
2020-08-01: 128.29361390938402
2020-08-02: 122.6776007716033
2

/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-c


2020-08-02: 0
2020-08-03: 0
2020-08-04: 0
2020-08-05: 0
2020-08-06: 0
2020-08-07: 0
2020-08-08: 0
2020-08-09: 0
2020-08-10: 0
2020-08-11: 0
2020-08-12: 0
2020-08-13: 0
2020-08-14: 0
2020-08-15: 0
2020-08-16: 0
2020-08-17: 0
2020-08-18: 0
2020-08-19: 0
2020-08-20: 0
2020-08-21: 0
2020-08-22: 0
2020-08-23: 0
2020-08-24: 0
2020-08-25: 0
2020-08-26: 0
2020-08-27: 0
2020-08-28: 0
2020-08-29: 0
2020-08-30: 0
2020-08-31: 0

Predicting for Belgium__nan
2020-08-01: 753.1722546018636
2020-08-02: 743.8559713241202
2020-08-03: 680.549791089049
2020-08-04: 664.7395725847664
2020-08-05: 771.6222360677823
2020-08-06: 879.8857328139652
2020-08-07: 1024.0838100528326
2020-08-08: 940.0217525224798
2020-08-09: 899.589712459102
2020-08-10: 933.8346563848411
2020-08-11: 932.8322376176013
2020-08-12: 1011.0662635855647
2020-08-13: 1082.8546737728966
2020-08-14: 1060.833503286845
2020-08-15: 1123.38232049741
2020-08-16: 838.1051847723757
2020-08-17: 926.2295020099727
2020-08-18: 805.338243122817
2020-08-19:

/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-c

2020-08-01: 114.99723779496969
2020-08-02: 94.90951946009716
2020-08-03: 79.50864139198069
2020-08-04: 63.990653372402306
2020-08-05: 61.98232938102399
2020-08-06: 60.612311125662075
2020-08-07: 41.5807083866798
2020-08-08: 24.242135579181152
2020-08-09: 0
2020-08-10: 0
2020-08-11: 0
2020-08-12: 0
2020-08-13: 0
2020-08-14: 0
2020-08-15: 0
2020-08-16: 0
2020-08-17: 0
2020-08-18: 0
2020-08-19: 0
2020-08-20: 0
2020-08-21: 0
2020-08-22: 0
2020-08-23: 0
2020-08-24: 0
2020-08-25: 0
2020-08-26: 0
2020-08-27: 0
2020-08-28: 0
2020-08-29: 0
2020-08-30: 0
2020-08-31: 0

Predicting for Belize__nan
2020-08-01: 16.32621517440272
2020-08-02: 23.86455860507293
2020-08-03: 31.01725961187337
2020-08-04: 37.2860779703435
2020-08-05: 43.399326323892
2020-08-06: 50.59224468670992
2020-08-07: 60.13100185507473
2020-08-08: 71.29086856548459
2020-08-09: 86.24131434221935
2020-08-10: 102.11968481362902
2020-08-11: 147.51413627394987
2020-08-12: 128.1682255183029
2020-08-13: 162.02918999581567
2020-08-14: 205.2

...


2020-08-20: 883.8597938293883
2020-08-21: 920.5499846229759
2020-08-22: 905.5809898863047
2020-08-23: 782.8097070123363
2020-08-24: 898.204623704796
2020-08-25: 932.4874003097573
2020-08-26: 912.3632691921832
2020-08-27: 836.8032975674982
2020-08-28: 864.8883079621387
2020-08-29: 870.7157512003294
2020-08-30: 802.5106879060422
2020-08-31: 878.3032449056914

Predicting for United States__Rhode Island
2020-08-01: 110.3670308635439
2020-08-02: 120.07954370193184
2020-08-03: 193.48787143098787
2020-08-04: 171.6553850073941
2020-08-05: 215.5331198656458
2020-08-06: 177.52061106359534
2020-08-07: 176.0346976442305
2020-08-08: 120.30411060179168
2020-08-09: 100.6614807849408
2020-08-10: 187.54517617033628
2020-08-11: 230.39390118906175
2020-08-12: 196.51819640851122
2020-08-13: 187.2738175117797
2020-08-14: 155.02764723815142
2020-08-15: 136.28207382487585
2020-08-16: 175.34349764730734
2020-08-17: 206.68570027015363
2020-08-18: 245.684858254679
2020-08-19: 186.9262611991516
2020-08-20: 177.

/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-c

2020-08-12: 135.67798854252425
2020-08-13: 121.71594827520033
2020-08-14: 208.24489059218115
2020-08-15: 221.46800738877158
2020-08-16: 250.68851998505124
2020-08-17: 237.79502230416392
2020-08-18: 284.57606937321617
2020-08-19: 300.61792420726334
2020-08-20: 339.75793863684726
2020-08-21: 382.1201340772776
2020-08-22: 338.6991920097449
2020-08-23: 363.9016751087515
2020-08-24: 418.5941168970974
2020-08-25: 429.545722596982
2020-08-26: 452.9267106241065
2020-08-27: 440.8370537124138
2020-08-28: 486.3934413090636
2020-08-29: 514.4730057492504
2020-08-30: 455.08225956201034
2020-08-31: 517.9954314464085

Predicting for United States__Washington
2020-08-01: 351.83709209336774
2020-08-02: 625.249252703047
2020-08-03: 586.4713755815615
2020-08-04: 440.10320128775595
2020-08-05: 430.17508345490023
2020-08-06: 442.5820326123748
2020-08-07: 196.9108576764743
2020-08-08: 264.6966849937425
2020-08-09: 379.3634826428782
2020-08-10: 349.9457794883978
2020-08-11: 286.3362083520071
2020-08-12: 267.8

/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-covid/competition/predictors/linear_reg/predict.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hist_cases_gdf = hist_cases_gdf[hist_cases_df.Date < current_date]
/Users/jackiedong/Documents/usc-xprize-c

2020-08-01: 9.34809403646981
2020-08-02: 13.646821100251096
2020-08-03: 17.33628568797574
2020-08-04: 20.68035804317467
2020-08-05: 24.183259241248855
2020-08-06: 27.944153908738855
2020-08-07: 32.86856755774048
2020-08-08: 39.21099153477624
2020-08-09: 44.863779745443225
2020-08-10: 50.630690002506356
2020-08-11: 56.40933621894439
2020-08-12: 61.934481135273074
2020-08-13: 67.47721361610195
2020-08-14: 73.14068337876998
2020-08-15: 79.37508186039062
2020-08-16: 86.2434692119123
2020-08-17: 92.49223641330278
2020-08-18: 98.84354060026396
2020-08-19: 104.03737189180845
2020-08-20: 109.43822927345093
2020-08-21: 115.02832635398497
2020-08-22: 122.16970631313188
2020-08-23: 128.4553069463681
2020-08-24: 133.74533885536178
2020-08-25: 139.21030509683305
2020-08-26: 144.00078981720125
2020-08-27: 148.50278046093385
2020-08-28: 153.33711603438587
2020-08-29: 198.94004202630197
2020-08-30: 157.9960381403363
2020-08-31: 149.04720900858183

Predicting for Yemen__nan
2020-08-01: 0
2020-08-02: 18

In [208]:
# Check the predictions
preds_df[preds_df['CountryName']=='United States'].tail()

,CountryName,RegionName,Date,PredictedDailyNewCases
70109,United States,Wyoming,2020-08-27,158.784379
70110,United States,Wyoming,2020-08-28,157.523226
70111,United States,Wyoming,2020-08-29,145.815875
70112,United States,Wyoming,2020-08-30,163.172781
70113,United States,Wyoming,2020-08-31,185.348631


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [36]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [1]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,0.0
Aruba,,2020-08-02,39.63190557658827
Aruba,,2020-08-03,35.080062326880025
Aruba,,2020-08-04,0.0
Afghanistan,,2020-08-01,128.29361390938394
Afghanistan,,2020-08-02,122.67760077160327
Afghanistan,,2020-08-03,156.69645813297407
Afghanistan,,2020-08-04,145.39971932182667
Angola,,2020-08-01,86.1982578696437


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [17]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [18]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [19]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
CPU times: user 28.1 ms, sys: 13.4 ms, total: 41.5 ms
Wall time: 1.5 s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

In [ ]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")